In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
pd.set_option('display.max_columns', None)

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
import string






import base64
import string
import re
from collections import Counter
from nltk.corpus import stopwords
stopwords = stopwords.words('english')


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from nltk.corpus import stopwords
import string 
import re

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

First we will load the dataset.

In [ ]:
df = pd.read_csv('/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv')
df.head()

In [ ]:
df = df.sort_values('title').reset_index(drop=True)
df.drop(['salary_range','job_id','department','benefits'],axis=1,inplace= True)
df['employment_type'] = df['employment_type'].bfill(axis = 0)
df['required_experience'] = df['required_experience'].bfill(axis=0)
df['required_education'] = df['required_education'].bfill(axis=0)
df['industry'] = df['industry'].bfill(axis = 0)
df['function'] = df['function'].bfill(axis = 0)

keep only the valid things and remove missing data that cannot be obtained.

In [ ]:
df = df[df['description'].notna()]
df = df.dropna(axis=0,how='any')

remove duplicate data from the dataset and keep only one instance of duplicate data.

In [ ]:
df = df.drop_duplicates(keep='first')

append requirements and company_profile to description column and drop them for better data processing

In [ ]:
df['description'] = df['description'] +' '+ df['requirements']+' '+df['company_profile']
df.drop(['company_profile','requirements'],axis=1,inplace=True);

seperate location to country_name and city

In [ ]:
import pycountry
df['city'] = df['location'].str.split(',',expand=True)[2]
df['country_code'] = df['location'].str.split(',',expand=True)[0]
df.loc[df['city']==' ','city'] = np.nan
df.dropna(inplace=True)
alpha2 = [x.alpha_2 for x in list(pycountry.countries)]
def getcountry(df):
    if df['country_code'] in alpha2:
        return pycountry.countries.get(alpha_2 = df['country_code']).name
df['country_name']=df.apply(getcountry,axis=1)

In [ ]:
df.drop(['location','country_code'],axis=1,inplace=True)

create a world cloud to get some insights about different types of job descriptions

In [ ]:
plt.figure(figsize = (20,20))
stopwords1 = set(STOPWORDS)
wc = WordCloud(background_color = "white", stopwords=stopwords1, width = 1600 , height = 800 , max_words = 3000).generate(" ".join(df[df.fraudulent == 0]['description']))
plt.axis("off")
plt.imshow(wc , interpolation = 'bilinear')
plt.savefig('no_fraud_cloud.jpeg')

In [ ]:
plt.figure(figsize = (20,20))
stopwords1 = set(STOPWORDS)
wc = WordCloud(background_color="white", stopwords=stopwords1, width = 1600 , height = 800 , max_words = 3000).generate(" ".join(df[df.fraudulent == 1]['description']))
plt.axis("off")
plt.imshow(wc , interpolation = 'bilinear')
plt.savefig('fraud_cloud.jpeg')

In [ ]:
df.to_csv("/kaggle/working/clean_data.csv")